In [41]:
import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv('dataset/train.csv')

In [43]:
df.shape

(36992, 25)

In [44]:
df.columns

Index(['customer_id', 'Name', 'age', 'gender', 'security_no',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'referral_id', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback', 'churn_risk_score'],
      dtype='object')

In [45]:
def _joined_through_ref_cleaner(row):
    yesOrNo = 'Yes' if row["referral_id"] != 'xxxxxxxx' else 'No'
    row['joined_through_referral'] = yesOrNo if (row['joined_through_referral'] == '?') else row['joined_through_referral']
    return row

def joined_through_ref_cleaner(df):
    df2 = df.apply(lambda row:_joined_through_ref_cleaner(row),axis=1)
    return df2
    
def churn_risk_score_negative(df):
    df.loc[(df["churn_risk_score"] < 0) | (df["churn_risk_score"] > 5)] = int(df["churn_risk_score"].mean())
    return df

def joining_date_converter(df):
    minDate = pd.to_datetime(df['joining_date']).min()
#     print(minDate)
    df["joining_date"] = (pd.to_datetime(df["joining_date"]) - minDate).astype('timedelta64[D]')
    return df

def last_visit_time_converter(df):
    df['last_visit_time'] = pd.to_timedelta(df['last_visit_time'], unit='ns').dt.total_seconds().astype('int64')
#     df["last_visit_time"] = pd.to_datetime(df["last_visit_time"]).astype('int64') // 10**9
    return df

def transform_data(df):
    df1 = joined_through_ref_cleaner(df)
    df2 = joining_date_converter(df1)
    df3 = last_visit_time_converter(df2)
    return df3

In [46]:
df = transform_data(df)
# df = churn_risk_score_negative(df)

In [47]:
def dropAllNA(df):
    df["gender"] = df["gender"].replace("Unknown",np.NaN)
    df["avg_frequency_login_days"] = df["avg_frequency_login_days"].replace("Error",np.NaN)
#     df["avg_frequency_login_days"] = df["avg_frequency_login_days"].replace("-999",50)
    df["medium_of_operation"] = df["medium_of_operation"].replace("?",np.NaN)
    
    df['gender'].fillna(df["gender"].value_counts().index[0],inplace=True)
    df['avg_frequency_login_days'].fillna(df[df["avg_frequency_login_days"] != '-999']['avg_frequency_login_days'].astype('float64').mean(),inplace=True)
    df['medium_of_operation'].fillna(df["medium_of_operation"].value_counts().index[0],inplace=True)
    df['region_category'].fillna(df["region_category"].value_counts().index[0],inplace=True)
    df['preferred_offer_types'].fillna(df["preferred_offer_types"].value_counts().index[0],inplace=True)
    
    df['points_in_wallet'].fillna(df["points_in_wallet"].astype('float64').mean(),inplace=True)
    df['points_in_wallet'] = df['points_in_wallet'].astype('float64')
    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].astype("float64")
    return df

In [48]:
df = dropAllNA(df)

In [49]:
df.isnull().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

In [50]:
df = df.fillna(df.mean())

In [51]:
df.isnull().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

In [52]:
df.shape

(36992, 25)

In [53]:
df.isnull().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

In [54]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [55]:
df.dtypes

customer_id                      object
Name                             object
age                               int64
gender                           object
security_no                      object
region_category                  object
membership_category              object
joining_date                    float64
joined_through_referral          object
referral_id                      object
preferred_offer_types            object
medium_of_operation              object
internet_option                  object
last_visit_time                   int64
days_since_last_login             int64
avg_time_spent                  float64
avg_transaction_value           float64
avg_frequency_login_days        float64
points_in_wallet                float64
used_special_discount            object
offer_application_preference     object
past_complaint                   object
complaint_status                 object
feedback                         object
churn_risk_score                  int64


In [56]:
df2 = df.select_dtypes(include="object")
df2 = df2.astype(str).apply(label_encoder.fit_transform)
df.dtypes

customer_id                      object
Name                             object
age                               int64
gender                           object
security_no                      object
region_category                  object
membership_category              object
joining_date                    float64
joined_through_referral          object
referral_id                      object
preferred_offer_types            object
medium_of_operation              object
internet_option                  object
last_visit_time                   int64
days_since_last_login             int64
avg_time_spent                  float64
avg_transaction_value           float64
avg_frequency_login_days        float64
points_in_wallet                float64
used_special_discount            object
offer_application_preference     object
past_complaint                   object
complaint_status                 object
feedback                         object
churn_risk_score                  int64


In [57]:
df2.head()

customer_id   Name  gender  security_no  region_category  \
0        32243  28675       0        34811                2   
1         7055  34951       0         5804                0   
2         5910  25315       0         1534                1   
3        32400  10462       1        32501                0   
4         6070  24704       0        29819                0   

   membership_category  joined_through_referral  referral_id  \
0                    3                        0        11358   
1                    4                        1         2225   
2                    2                        1          471   
3                    2                        1         5506   
4                    2                        0        11358   

   preferred_offer_types  medium_of_operation  internet_option  \
0                      1                    1                2   
1                      1                    1                1   
2                      1                    1                2   
3                      1                    1                1   
4                      0                    2                1   

   used_special_discount  offer_application_preference  past_complaint  \
0                      1                             1               0   
1                      1                             0               1   
2                      0                             1               1   
3                      0                             1               1   
4                      0                             1               1   

   complaint_status  feedback  
0                 1         4  
1                 2         5  
2                 3         3  
3                 4         3  
4                 2         3

In [58]:
from sklearn.model_selection import train_test_split
X = df2[['gender',
       'region_category', 'membership_category', 
       'joined_through_referral', 'preferred_offer_types',
       'medium_of_operation', 'internet_option',
        'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback']]
X[['points_in_wallet', 'age', 'avg_time_spent', 'avg_frequency_login_days','last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value']] = df[['points_in_wallet', 'age', 'avg_time_spent','avg_frequency_login_days', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value']]
y = df["churn_risk_score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# print(X.shape,y.shape)

C:\Users\MAYANK\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [59]:
X.head()

gender  region_category  membership_category  joined_through_referral  \
0       0                2                    3                        0   
1       0                0                    4                        1   
2       0                1                    2                        1   
3       1                0                    2                        1   
4       0                0                    2                        0   

   preferred_offer_types  medium_of_operation  internet_option  \
0                      1                    1                2   
1                      1                    1                1   
2                      1                    1                2   
3                      1                    1                1   
4                      0                    2                1   

   used_special_discount  offer_application_preference  past_complaint  \
0                      1                             1               0   
1                      1                             0               1   
2                      0                             1               1   
3                      0                             1               1   
4                      0                             1               1   

   complaint_status  feedback  points_in_wallet  age  avg_time_spent  \
0                 1         4        781.750000   18          300.63   
1                 2         5        686.882199   32          306.34   
2                 3         3        500.690000   44          516.16   
3                 4         3        567.660000   37           53.27   
4                 2         3        663.060000   31          113.13   

   avg_frequency_login_days  last_visit_time  joining_date  \
0                      17.0            58082         959.0   
1                      10.0            45493         970.0   
2                      22.0            82401         680.0   
3                       6.0            57470         667.0   
4                      16.0            56804         985.0   

   days_since_last_login  avg_transaction_value  
0                     17               53005.25  
1                     16               12838.38  
2                     14               21027.00  
3                     11               25239.56  
4                     20               24483.66

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
model = RandomForestClassifier()
# model = AdaBoostClassifier()
# model = KNeighborsClassifier()
# model = DecisionTreeClassifier()

In [61]:
y_train.isna().sum()

0

In [62]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [63]:
y_pred = model.predict(X_test)

In [64]:
from sklearn.metrics import f1_score,mean_squared_error
score = f1_score(y_test, y_pred,average="macro")
score

0.6219734233041044

In [65]:
result_df = pd.DataFrame({
    "churn_risk_score":y_pred
})

In [66]:
result_df["churn_risk_score"].unique()

array([1, 4, 3, 5, 2], dtype=int64)

In [67]:
real_test_data = pd.read_csv("dataset/test.csv")
real_test_data.head()

customer_id              Name  age gender  \
0  fffe43004900440031003700300030003400    Alethia Meints   50      F   
1  fffe43004900440031003900370037003300        Ming Lopez   41      M   
2  fffe43004900440034003800360037003000  Carina Flannigan   31      F   
3  fffe43004900440036003200370033003400      Kyung Wanner   64      M   
4  fffe43004900440035003000370031003900       Enola Gatto   16      F   

  security_no region_category membership_category joining_date  \
0     OQJ1XAY         Village  Premium Membership   2015-11-02   
1     OUQRPKO         Village     Gold Membership   2016-03-01   
2     02J2RE7            Town   Silver Membership   2017-03-03   
3     5YEQIF1            Town   Silver Membership   2017-08-18   
4     100RYB5            Town       No Membership   2015-05-05   

  joined_through_referral referral_id  ... days_since_last_login  \
0                      No    xxxxxxxx  ...                    12   
1                      No    xxxxxxxx  ...                    11   
2                      No    xxxxxxxx  ...                    18   
3                     Yes     CID8941  ...                  -999   
4                     Yes     CID5690  ...                     6   

  avg_time_spent avg_transaction_value avg_frequency_login_days  \
0         386.26              40721.44                      7.0   
1          37.80               9644.40                      9.0   
2         215.36               3693.25                     21.0   
3          44.57              36809.56                     11.0   
4         349.88              40675.86                      8.0   

   points_in_wallet  used_special_discount  offer_application_preference  \
0        733.830000                    Yes                            No   
1        726.000000                    Yes                            No   
2        713.780000                    Yes                            No   
3        744.970000                    Yes                            No   
4        299.048351                     No                           Yes   

  past_complaint          complaint_status              feedback  
0             No            Not Applicable  Poor Product Quality  
1             No            Not Applicable          Poor Website  
2            Yes       Solved in Follow-up   No reason specified  
3            Yes  No Information Available          Too many ads  
4            Yes       Solved in Follow-up          Poor Website  

[5 rows x 24 columns]

In [68]:
real_test_data = transform_data(real_test_data)
real_test_data = dropAllNA(real_test_data)
real_test_data2 = real_test_data[['age', 'gender',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback']]

In [69]:
real_test_data2 = real_test_data2.fillna(real_test_data2.mean())

In [70]:
real_test_data2.dtypes

age                               int64
gender                           object
region_category                  object
membership_category              object
joining_date                    float64
joined_through_referral          object
preferred_offer_types            object
medium_of_operation              object
internet_option                  object
last_visit_time                   int64
days_since_last_login             int64
avg_time_spent                  float64
avg_transaction_value           float64
avg_frequency_login_days        float64
points_in_wallet                float64
used_special_discount            object
offer_application_preference     object
past_complaint                   object
complaint_status                 object
feedback                         object
dtype: object

In [71]:
encoded_str = real_test_data2.select_dtypes(include="object")#.apply(label_encoder.fit_transform)

In [72]:
model_ip = encoded_str.astype(str).apply(label_encoder.fit_transform)

In [73]:
model_ip[['points_in_wallet', 'age', 'avg_time_spent', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value','avg_frequency_login_days']] = real_test_data2[['points_in_wallet', 'age', 'avg_time_spent', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value','avg_frequency_login_days']].astype(str)
model_ip[['last_visit_time', 'joining_date']] = real_test_data2[['last_visit_time', 'joining_date']]
model_ip.head()

gender  region_category  membership_category  joined_through_referral  \
0       0                2                    4                        0   
1       1                2                    1                        0   
2       0                1                    5                        0   
3       1                1                    5                        1   
4       0                1                    2                        1   

   preferred_offer_types  medium_of_operation  internet_option  \
0                      2                    2                2   
1                      2                    1                0   
2                      1                    0                1   
3                      0                    1                0   
4                      2                    2                1   

   used_special_discount  offer_application_preference  past_complaint  \
0                      1                             0               0   
1                      1                             0               0   
2                      1                             0               1   
3                      1                             0               1   
4                      0                             1               1   

   complaint_status  feedback    points_in_wallet age avg_time_spent  \
0                 1         2              733.83  50         386.26   
1                 1         3               726.0  41           37.8   
2                 3         0              713.78  31         215.36   
3                 0         7              744.97  64          44.57   
4                 3         3  299.04835123722256  16         349.88   

   last_visit_time  joining_date days_since_last_login avg_transaction_value  \
0            26370         305.0                    12              40721.44   
1            80476         425.0                    11                9644.4   
2            60039         792.0                    18               3693.25   
3            53777         960.0                  -999              36809.56   
4            10673         124.0                     6              40675.86   

  avg_frequency_login_days  
0                      7.0  
1                      9.0  
2                     21.0  
3                     11.0  
4                      8.0

In [74]:
model_ip[['last_visit_time', 'joining_date']].isna().sum()

last_visit_time    0
joining_date       0
dtype: int64

In [75]:
model_ip.dtypes

gender                            int32
region_category                   int32
membership_category               int32
joined_through_referral           int32
preferred_offer_types             int32
medium_of_operation               int32
internet_option                   int32
used_special_discount             int32
offer_application_preference      int32
past_complaint                    int32
complaint_status                  int32
feedback                          int32
points_in_wallet                 object
age                              object
avg_time_spent                   object
last_visit_time                   int64
joining_date                    float64
days_since_last_login            object
avg_transaction_value            object
avg_frequency_login_days         object
dtype: object

In [76]:
# str_needed = real_test_data.select_dtypes(exclude=["float64","int64"])
real_test_pred = model.predict(model_ip)

In [77]:
final_result_df = pd.DataFrame({
    "customer_id":real_test_data["customer_id"],
    "churn_risk_score":real_test_pred.astype(int)
})

In [78]:
final_result_df.head()

customer_id  churn_risk_score
0  fffe43004900440031003700300030003400                 3
1  fffe43004900440031003900370037003300                 3
2  fffe43004900440034003800360037003000                 4
3  fffe43004900440036003200370033003400                 3
4  fffe43004900440035003000370031003900                 5

In [79]:
final_result_df.to_csv(f"try-score-{score}.csv")

In [80]:
# import seaborn as sns
# sns.pairplot(model_ip)